This is the sourch code of our data split technique based on clustering and stratified sampling.
You can modify the code according to your needs, including the file path and partition scale.

In [ ]:
from rdkit import Chem
from rdkit.Chem import MACCSkeys
from rdkit import DataStructs
from rdkit.Chem import rdMolDescriptors,MolStandardize,AllChem,rdMolDescriptors

import pandas as pd
import numpy as np

import glob
import os 

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [ ]:
data = np.array(pd.read_csv('**/new_RE.csv'))  # put your own data path
smiles = list(data[:,0])

In [ ]:
def smiles2maccs(smi_list: list) -> np.array:
    MACCS = []
    for smi in smi_list:
        m = Chem.MolFromSmiles(smi)
        fps = MACCSkeys.GenMACCSKeys(m) 
        MACCS.append(fps)
    MACCS = np.array(MACCS)
    return MACCS
MACCS = smiles2maccs(smiles)

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=8).fit(MACCS)
cluster = kmeans.predict(MACCS)
zone1_idx , zone2_idx , zone3_idx , zone4_idx , zone5_idx ,zone6_idx ,zone7_idx ,zone8_idx = [],[],[],[],[],[],[],[]
for i in range(cluster.shape[0]):
    if cluster[i]==0:
        zone1_idx.append(i)
    if cluster[i]==1:
        zone2_idx.append(i)
    if cluster[i]==2:
        zone3_idx.append(i)
    if cluster[i]==3:
        zone4_idx.append(i)
    if cluster[i]==4:
        zone5_idx.append(i)
    if cluster[i]==5:
        zone6_idx.append(i)
    if cluster[i]==6:
        zone7_idx.append(i)
    if cluster[i]==7:
        zone8_idx.append(i)
##   Assume that no terminal cluster exists

In [ ]:
def get_similarity_matrix(smi_list):
    fps_list = []
    for smi in smiles:
        fps = MACCSkeys.GenMACCSKeys(Chem.MolFromSmiles(smi))
        fps_list.append(fps)

    fps_array = np.zeros((len(smi_list),len(smi_list)))
    
    for i in range(len(fps_list)):
        for j in range(len(fps_list)):
            s = DataStructs.FingerprintSimilarity(fps_list[i],fps_list[j],metric=DataStructs.TanimotoSimilarity)
            fps_array[i][j] = s
            fps_array[j][i] = s
    return fps_array

fps_array = get_similarity_matrix(smiles)

In [ ]:
def get_cc_ml(fps_copy,zone_idx):  
    fps = fps_copy[zone_idx,:]
    fps = fps[:,zone_idx]
    cluster_center = [] 
    molecular = []     
    smiles_idx = [x for x in range(len(zone_idx))] 

    cluster_center_idx = random.choice(smiles_idx)
    random_column = fps[:, cluster_center_idx]
    indices_list = list(np.where(random_column > *)[0])      # * denotes to the average of similarity , you need to calculate similarity index of the whole dataset

    cluster_center.append(cluster_center_idx)
    molecular.append(indices_list)
    smiles_idx = [i for i in smiles_idx if i not in indices_list]    
                     
    while len(smiles_idx) != 0:
        cluster_center_idx = random.choice(smiles_idx)                                    
        smiles_idx.remove(cluster_center_idx)                                              
        indices_list = []
        random_column = fps[:, cluster_center_idx]
        threshold_random = random_column[random_column > *]                        
        threshold_indices = np.where(random_column > *)[0].tolist()                 
        for i in range(0,len(cluster_center)):                                       
            compare_column = fps[:, cluster_center[i]]                               
            compare_column = compare_column[threshold_indices]                           
            indices_pre = np.where(threshold_random > compare_column)[0].tolist()          
            indices = [threshold_indices[idx] for idx in indices_pre]                      
            molecular[i] = [x for x in molecular[i] if x not in cluster_center]           
            molecular[i] = [x for x in molecular[i] if x not in indices]                   
            indices_list.append(indices)
        s = set(indices_list[0])        
        for i in range(1, len(indices_list)):
            s = s & set(indices_list[i]) 
        smiles_idx = [i for i in smiles_idx if i not in s]       
        molecular.append(list(s))
        cluster_center.append(cluster_center_idx)
    return cluster_center,molecular

In [ ]:
import random
random.seed()        #choose the different random seeds

In [ ]:
cluster_center1,molecular1 = get_cc_ml(fps_array,zone1_idx)
cluster_center2,molecular2 = get_cc_ml(fps_array,zone2_idx)
cluster_center3,molecular3 = get_cc_ml(fps_array,zone3_idx)
cluster_center4,molecular4 = get_cc_ml(fps_array,zone4_idx)
cluster_center5,molecular5 = get_cc_ml(fps_array,zone5_idx)
cluster_center6,molecular6 = get_cc_ml(fps_array,zone6_idx)
cluster_center7,molecular7 = get_cc_ml(fps_array,zone7_idx)
cluster_center8,molecular8 = get_cc_ml(fps_array,zone8_idx)

In [ ]:
def get_aboslute_idx(zone_idx,cluster_center,molecular):        
    relative_idx = []
    ab_idx = []
    for j in range(len(cluster_center)):
        cluster = sum([[cluster_center[j]],molecular[j]],[])
        relative_idx.append(cluster)
    for l in relative_idx:
        cluster_ab = []
        if len(l) != 0:
            for i in l:
                cluster_ab.append(zone_idx[i])
        else:
            pass
        ab_idx.append(sorted(cluster_ab))
    return ab_idx

In [ ]:
cluster1 = get_aboslute_idx(zone1_idx,cluster_center1,molecular1)
cluster2 = get_aboslute_idx(zone2_idx,cluster_center2,molecular2)
cluster3 = get_aboslute_idx(zone3_idx,cluster_center3,molecular3)
cluster4 = get_aboslute_idx(zone4_idx,cluster_center4,molecular4)
cluster5 = get_aboslute_idx(zone5_idx,cluster_center5,molecular5)
cluster6 = get_aboslute_idx(zone6_idx,cluster_center6,molecular6)
cluster7 = get_aboslute_idx(zone7_idx,cluster_center7,molecular7)
cluster8 = get_aboslute_idx(zone8_idx,cluster_center8,molecular8)

In [ ]:
def get_atom_num(ab_idx,smiles):
    zone1_list,zone2_list,zone3_list,zone4_list,zone5_list,zone6_list,zone7_list,zone8_list,zone9_list,zone10_list,zone11_list = [],[],[],[],[],[],[],[],[],[],[]
    for idx in ab_idx:
        smi = smiles[idx]
        m = Chem.AddHs(Chem.MolFromSmiles(smi))
        c = m.GetNumAtoms()
        if c >= 0 and c < 10:
            zone1_list.append(idx)
        elif c >= 10 and c < 20:
            zone2_list.append(idx)
        elif c >= 20 and c < 30:
            zone3_list.append(idx)
        elif c >= 30 and c < 40:
            zone4_list.append(idx)
        elif c >= 40 and c < 50:
            zone5_list.append(idx)
        elif c >= 50 and c < 60:
            zone6_list.append(idx)
        elif c >= 60 and c < 70:
            zone7_list.append(idx)
        elif c >= 70 and c < 80:
            zone8_list.append(idx)
        elif c >= 80 and c < 90:
            zone9_list.append(idx)
        elif c >= 90 and c < 100:
            zone10_list.append(idx)
        elif c == 100:
            zone11_list.append(idx)
    atom_count_list = [zone1_list,zone2_list,zone3_list,zone4_list,zone5_list,zone6_list,zone7_list,zone8_list,zone9_list,zone10_list,zone11_list]
    return atom_count_list

In [ ]:
def get_rotate_num(ab_idx,smiles):
    r1_list,r2_list,r3_list,r4_list,r5_list,r6_list,r7_list,r8_list,r9_list,r10_list,r11_list,r12_list,r13_list = [],[],[],[],[],[],[],[],[],[],[],[],[]
    for idx in ab_idx:
        smi = smiles[idx]
        m = Chem.AddHs(Chem.MolFromSmiles(smi))
        num = rdMolDescriptors.CalcNumRotatableBonds(m)
        if num == 0:
            r1_list.append(idx)
        elif num ==1:
            r2_list.append(idx)
        elif num == 2:
            r3_list.append(idx)
        elif num == 3:
            r4_list.append(idx)
        elif num == 4:
            r5_list.append(idx)
        elif num == 5:
            r6_list.append(idx)
        elif num == 6:
            r7_list.append(idx)
        elif num == 7:
            r8_list.append(idx)
        elif num == 8:
            r9_list.append(idx)
        elif num == 9:
            r10_list.append(idx)
        elif num == 10:
            r11_list.append(idx)
        elif num == 11:
            r12_list.append(idx)
        elif num == 12:
            r13_list.append(idx)
    rotate_count_list = [r1_list,r2_list,r3_list,r4_list,r5_list,r6_list,r7_list,r8_list,r9_list,r10_list,r11_list,r12_list,r13_list]
    return rotate_count_list

In [ ]:
def get_ring_num(ab_idx,smiles):
    ring1_list,ring2_list,ring3_list,ring4_list,ring5_list,ring6_list,ring7_list,ring8_list,ring9_list,ring10_list,ring11_list,ring12_list,ring13_list,ring14_list,ring15_list,ring16_list,ring17_list,ring18_list,ring19_list,ring20_list,ring21_list,ring22_list,ring23_list,ring24_list,ring25_list,ring26_list,ring27_list,ring28_list,ring29_list = [],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]
    for idx in ab_idx:
        smi = smiles[idx]
        m = Chem.AddHs(Chem.MolFromSmiles(smi))
        num = m.GetRingInfo().NumRings()
        if num == 0:
            ring1_list.append(idx)
        elif num ==1:
            ring2_list.append(idx)
        elif num == 2:
            ring3_list.append(idx)
        elif num == 3:
            ring4_list.append(idx)
        elif num == 4:
            ring5_list.append(idx)
        elif num == 5:
            ring6_list.append(idx)
        elif num == 6:
            ring7_list.append(idx)
        elif num == 7:
            ring8_list.append(idx)
        elif num == 8:
            ring9_list.append(idx)
        elif num == 9:
            ring10_list.append(idx)
        elif num == 10:
            ring11_list.append(idx)
        elif num == 11:
            ring12_list.append(idx)
        elif num == 12:
            ring13_list.append(idx)
        elif num == 13:
            ring14_list.append(idx)
        elif num == 14:
            ring15_list.append(idx)
        elif num == 15:
            ring16_list.append(idx)
        elif num == 16:
            ring17_list.append(idx)
        elif num == 17:
            ring18_list.append(idx)
        elif num == 18:
            ring19_list.append(idx)
        elif num == 19:
            ring20_list.append(idx)
        elif num == 20:
            ring21_list.append(idx)
        elif num == 21:
            ring22_list.append(idx)
        elif num == 22:
            ring23_list.append(idx)
        elif num == 23:
            ring24_list.append(idx)
        elif num == 24:
            ring25_list.append(idx)
        elif num == 25:
            ring26_list.append(idx)
        elif num == 26:
            ring27_list.append(idx)
        elif num == 27:
            ring28_list.append(idx)
        elif num == 28:
            ring29_list.append(idx)
    
    rotate_count_list = [ring1_list,ring2_list,ring3_list,ring4_list,ring5_list,ring6_list,ring7_list,ring8_list,ring9_list,ring10_list,ring11_list,ring12_list,ring13_list,ring14_list,ring15_list,ring16_list,ring17_list,ring18_list,ring19_list,ring20_list,ring21_list,ring22_list,ring23_list,ring24_list,ring25_list,ring26_list,ring27_list,ring28_list,ring29_list]
    return rotate_count_list

In [ ]:
def cal_symbol(mol):
    symbol = []
    m = Chem.RemoveHs(mol)
    for atom in mol.GetAtoms():
        symbol.append(atom.GetSymbol())
    symbol = set(symbol)
    return list(symbol)
def get_symbol_layer(ab_idx,smiles):
    O_list ,Br_list ,Cl_list ,Te_list ,B_list ,S_list ,N_list ,Se_list ,F_list ,As_list ,I_list ,Si_list ,P_list,C_list = [],[],[],[],[],[],[],[],[],[],[],[],[],[]
    symbol_list = []
    for idx in ab_idx:
        mol = Chem.MolFromSmiles(smiles[idx])
        symbol = cal_symbol(mol)        
        if 'Te' in symbol:         
            Te_list.append(idx)
            continue
        if 'I' in symbol:
            I_list.append(idx)
            continue
        if 'Br' in symbol:
            Br_list.append(idx)
            continue
        if 'Se' in symbol:
            Se_list.append(idx)
            continue
        if 'As' in symbol:
            As_list.append(idx)
            continue
        if 'Cl' in symbol:
            Cl_list.append(idx)
            continue
        if 'S' in symbol:
            S_list.append(idx)
            continue
        if 'P' in symbol:
            P_list.append(idx)
            continue
        if 'Si' in symbol:
            Si_list.append(idx)
            continue
        if 'F' in symbol:
            F_list.append(idx)
            continue
        if 'O' in symbol:
            O_list.append(idx)
            continue
        if 'N' in symbol:
            N_list.append(idx)
            continue
        if 'B' in symbol:
            B_list.append(idx)
            continue
        if 'C' in symbol:
            C_list.append(idx)
    symbol_all_list = [O_list ,Br_list ,Cl_list ,Te_list ,B_list ,S_list ,N_list ,Se_list ,F_list ,As_list ,I_list ,Si_list ,P_list , C_list]    
    return symbol_all_list

In [ ]:
def sampling(cluster_list):
    sample_idx = []
    for c in cluster_list:    
        c_atom_cluster = get_atom_num(c,smiles)    
        for b in c_atom_cluster:  
            c_rotate_cluster = get_rotate_num(b,smiles)
            for a in c_rotate_cluster:
                c_ring_cluster = get_ring_num(a,smiles) 
                for i in c_ring_cluster:    
                    c_rotate_clu = get_symbol_layer(i,smiles)     
                    for j in c_rotate_clu:             
                        if len(j)!=0:
                            sample_idx.append(random.sample(j,int(len(j)* %) if int(len(j)* %) != 0 else 1))      # % denotes to the ratio that you prefer to adpot , you need to choose your own value.
                        else:
                            pass
    return sample_idx

In [ ]:
sampling1 = sum(sampling(cluster1),[])
sampling2 = sum(sampling(cluster2),[])
sampling3 = sum(sampling(cluster3),[])
sampling4 = sum(sampling(cluster4),[])
sampling5 = sum(sampling(cluster5),[])
sampling6 = sum(sampling(cluster6),[])
sampling7 = sum(sampling(cluster7),[])
sampling8 = sum(sampling(cluster8),[])
t1 = [x for x in zone1_idx if x not in sampling1]
t2 = [x for x in zone2_idx if x not in sampling2]
t3 = [x for x in zone3_idx if x not in sampling3]
t4 = [x for x in zone4_idx if x not in sampling4]
t5 = [x for x in zone5_idx if x not in sampling5]
t6 = [x for x in zone6_idx if x not in sampling6]
t7 = [x for x in zone7_idx if x not in sampling7]
t8 = [x for x in zone8_idx if x not in sampling8]
train = sampling1 + sampling2 + sampling3 + sampling4 + sampling5+ sampling6 + sampling7 + sampling8
test = t1 + t2 + t3 + t4 + t5 + t6 + t7 +t8

In [ ]:
train_data ,test_data = data[train,:] , data[test,:]
pd.DataFrame(train_data).to_csv('**/train.csv',index=False) # put your own data path
pd.DataFrame(test_data).to_csv('**/test.csv',index=False) # put your own data path